# The Unappealingly Titled Project H2O

- Python version (see folder for R code)
- work in progress

## Objective

We are primarily interested in participats' ability to discriminate between tap and potted water of some brand on average. In the second place, we are interested in assessing individual's ability to identify the sampled water conditional on their confidence.

## Methods

The sampling process will span up to two months owing to constraints. The aim is to collect the largest sample attainable in up to five rounds per participant. See Sample collection below.

The analysis will consist of three parts. We shall assess (i) the collective ability in each round using the A/B test and the exact Fisher test, (ii) the collective ability across rounds using generalised estimating equations, and (iii) the individual ability across rounds using the Brier score.

### Frequentist estimates

The frequentist estimates are simple proportions of counts $n_{k1}$ in the marginal group counts $n_k$.

|           | success (guessed potted water) $(g = 1)$ | failure (guessed tap water) $(g = 2)$ | sample margin |
| --------- | ------- | ------- | ----- |
| **treatment (sampled potted water)** $(s = 1)$ | $n_{11}$     | $n_{12}$     | $n_{s_p}=n_{11}+n_{12}$ |
| **control (sampled tap water)** $(s = 2)$ | $n_{21}$     | $n_{22}$     | $n_{s_t}=n_{21}+n_{22}$ |
| **guess margin** | $n_{g_p}=n_{11}+n_{21}$   | $n_{g_t}=n_{12}+n_{22}$   | $N=n_{11}+n_{12}+n_{21}+n_{22}$   |

Then, estimators $\hat{p}_1 = \frac{1}{n_{s_p}} \sum_{i=1}^{n_{s_p}} 1_{\{g_i = 1\}} = \frac{n_{11}}{n_{s_p}}$ and $\hat{p}_2 = \frac{1}{n_{s_t}} \sum_{i=1}^{n_{s_t}} 1_{\{g_i = 1\}} = \frac{n_{21}}{n_{s_t}}$ solve the Bernoulli likelihood $\mathcal{L}(p_k;y_k)=\prod_{i=1}^{n_k}p_k^{y_{ki}}(1-p_k)^{1-y_{ki}}=p_k^{n_{k1}}(1-p_k)^{n_k-n_{k1}}$ where $n_{k1}$ is the count of successes in $n_{k}$ trials in sample group $k \in \{1,2\}=\{s_p,s_t\}$ and individual outcomes $y_{ki}\in \{0,1\}=\{t,p\}$.

#### Risk and odds ratios

| measure | estimator | null |
|---------|-----------|------|
| risk ratio | $\rho=\frac{p_1}{p_2}$ | $H_0:\rho=\rho_0=1$ |
| odds ratio | $\theta=\frac{p_1/(1-p_1)}{p_2/(1-p_2)}$ | $H_0:\theta=\theta_0=1$ |

##### Risk ratio confidence intervals

$$
\hat \rho = \frac{\hat p_1}{\hat p_2}, \qquad
\log \hat \rho = \log(\hat p_1) - \log(\hat p_2).
$$

$$
\widehat{\text{SE}}(\log \hat \rho)
= \sqrt{\frac{1 - \hat p_1}{\hat p_1 n_{s_p}} + \frac{1 - \hat p_2}{\hat p_2 n_{s_t}}}=\sqrt{\frac{1}{n_{11}} - \frac{1}{n_{11}+n_{12}} + \frac{1}{n_{21}} - \frac{1}{n_{21}+n_{22}}}
$$

$$
\rho \in \exp{\left( \ln{\hat{\rho}}\pm Z_{1-\alpha/2}\sqrt{\widehat{\text{SE}}(\ln{\hat{\rho}})} \right)}
$$

##### Odds ratio confidende intervals

$$
\hat \theta = \frac{n_{11} n_{22}}{n_{21} n_{12}}, \quad
\log \hat \theta = \log n_{11} + \log n_{22} - \log n_{12} - \log n_{21},
$$

$$
\widehat{\text{SE}}(\ln{\hat{\theta}})
= \sqrt{\frac{1}{n_{11}}+\frac{1}{n_{12}}+\frac{1}{n_{21}}+\frac{1}{n_{22}}}.
$$

Note that we commonly add $0.5$ to each $n_{kj}$ to avoid division by zero.

$$
\theta \in \exp{\left( \ln{ \hat \theta \pm Z_{1-\alpha/2}\ \widehat{\mathrm{SE}}(\ln{\hat{\theta}}) } \right)}
$$

### Bayesian estimates

We shall use a standard beta-binomial setup with conjugate priors. This is generally summarised with the continuous form of the Bayes rule like so,

$$
f(\theta \mid y) = \frac{\prod_{i=1}^{n_k} g(y \mid \theta) f(\theta)}{\int g(y \mid \theta) f(\theta) \, d\theta}
$$

We set up the model like so, 

- $y_k \mid p_k \sim \text{binomial}(n_k, p_k), \text{for } k=1,2$ (likelihood)
- $p_k \sim \text{Beta}(\alpha, \beta)$, choose non-informative $p_k \sim \text{Beta}(\frac{1}{2}, \frac{1}{2})$ (Jeffreys prior) 
- $p_k \mid y_k \sim \text{Beta}(y_k+\alpha,n_k-y_k+\beta) = \text{Beta}(\frac{1}{2}+y_k,\frac{1}{2}+n_k-y_k)$ (implied posterior)

Hence the posterior,

$$
f(p_k \mid y_k) \propto \mathcal{L}(p_k;y_k) f(p_k) = \left(p_k^{y_k}(1-p_k)^{n_k-y_k}\right)\left(p_k^{\frac{1}{2}-1}(1-p_k)^{\frac{1}{2}-1}\right) = p_k^{y_k-\frac{1}{2}}(1-p_k)^{n_k-y_k-\frac{1}{2}}
$$

We shall run 16e4 simulations with an additional 4e4 burnin in six chains, and we are interested in the median and the 95% credible interval. We use the estimates to compute the risk and odds ratios and the respective credible intervals.

### Generalised estimating equations

Generalised estimating equations approximate the (marginal) population-average effect for our clustered sample of $i=1,...,M$ individuals across $j=1,...,5$ rounds. GEE apply robust standard errors to allow valid inference.

The results are interpreted as a binomial logit model like so,

$$
\text{logit}\left(\pi_{ij}\right)=\ln{\left(\frac{\pi_{ij}}{1-\pi_{ij}}\right)}=\beta_0+\beta_1 x_{ij}
$$

Where

- $\pi_{ij}=P(Y_{ij}=1)=\mathbb{E}\left[Y_{ij} \mid X_{ij} \right]=\text{logit}^{-1}(X_{ij}'\beta)$ is the probability that participant $i$ gets predicts potted water ($g=1$) in round $j$
- $\frac{\pi_{ij}}{1-\pi_{ij}}$ is the odds of predicting potted water ($g=1$)
- $\beta_0$ is the log-odds when $x_{ij} = 0$ (intercept)
- $\beta_1$ is the effect of $x_{ij} = 1$ on the log-odds

GEE solve the following gradient for $\beta$,

$$
U(\beta)=\sum_{i=1}^N D_i'V_i^{-1}(y_i-\mu_i)=0
$$

Where $D_i=\frac{\partial \pi_i}{\partial \beta'}=A_i X_i$ with $A_i = \text{diag}\left(v(\mu_{it})\right)$ and the working covariance,

$$
V_i = \phi A_i^{1/2},R(\alpha),A_i^{1/2},\quad
A_i=\text{diag}\,\left(\pi_{ij}(1-\pi_{ij})\right)
$$

Where $R(\alpha_i)=\left[\begin{matrix} 1 & \alpha & \alpha & \dots & \alpha \\ \alpha & 1 & \alpha & \dots & \alpha \\
\vdots & \vdots & \ddots & \vdots & \vdots \\ \alpha & \alpha & \dots & 1\end{matrix}\right]$ is the exchangeable working correlation structure. No closed form exists. The algorithm solves the system iteratively.

#### Retrieve probabilities

$$
\pi_0=\text{logit}^{-1}(\beta_0)=\frac{1}{1+e^{-\beta_0}}=1-\frac{1}{1+e^{\beta_0}}=\text{plogis}(\beta_0)
$$

$$
\pi_1=\text{logit}^{-1}(\beta_0 + \beta_1)=\frac{1}{1+e^{-(\beta_0+\beta_1)}}=1-\frac{1}{1+e^{\beta_0+\beta_1}}=\text{plogis}(\beta_0 + \beta_1)
$$

#### Relative risk

$$
\hat{\rho}=\frac{\hat{\pi}_1}{\hat{\pi}_0}
$$

$$
\ln{(\hat{\rho})}=\ln{\left(\frac{\pi_1(\beta)}{\pi_0(\beta)}\right)}=\ln{\left(\frac{\text{logit}^{-1}(\beta_0+\beta_1)}{\text{logit}^{-1}(\beta_0)}\right)}
$$

With confidence intervals,

$$
\rho \in \exp{\left(\ln{\hat{\rho}}\pm Z_{1-\alpha/2}\,\widehat{\text{SE}}(\ln{\hat{\rho}})\right)}, \quad
\text{Var}\left(\ln{\hat{\rho}}\right)\approx g'\widehat{\text{Var}}_{\text{gp}}(\hat{\beta})g, \quad
g=\left[\begin{matrix}\frac{\partial\ln{\hat{\rho}}}{\partial\beta_0} \\ \frac{\partial\ln{\hat{\rho}}}{\partial\beta_1}\end{matrix}\right]=\left[\begin{matrix}\hat{\pi}_0-\hat{\pi}_1 \\ 1-\hat{\pi}_1\end{matrix}\right]
$$

#### Odds ratio

$$
\hat{\theta}=\frac{\hat{\pi}_1/1-\hat{\pi}_1}{\hat{\pi}_0/1-\hat{\pi}_0}
$$

With confidence intervals,

$$
\theta \in \exp{\left(\ln{\hat{\theta}}\pm Z_{1-\alpha/2}\,\text{SE}(\ln{\hat{\beta}})\right)}, \quad \text{SE}(\ln{\hat{\beta}})=\text{SE}(\ln{\hat{\theta}})
$$

### Brier score

The typical Brier score for a set of clustered outcomes and predictions for $j=1,...,J$ measures the accuracy of the participant's predictions as follows,

$$
\text{BS}=\frac{1}{n}\sum_{j=1}^J (P_j - 1_{\{X_j = 1\}})^2
$$

Where $P_j \in [0,1]$ is a measure of the participant's confidence in the outcome $X_j \in \{0,1\}$. Particularly $P_j \to 1$ implies higher confidence in $X_j = 1$ and $P_j \to 0$ implies a lower confidence in $X_j = 1$. Lower $BS$ imply a higher accuracy. See Brier (1950).

This can be reconceived as a measure of confidence like so,

$$
\text{BS}=\frac{1}{n}\sum_{j=1}^J (Z_j - 1_{\{S_j = 1\}})^2, \quad
S_j =
\begin{cases}
1, & X_j = Y_j,\\
0, & X_j \neq Y_j
\end{cases}
$$

Where $X_j \in {\{0,1\}}$ denotes the outcome, $Y_j \in {\{0,1\}}$ denotes the categorical prediction, $S_j \in {\{0,1\}}$ is the correctnes indicator and $Z_j \in [0,1]$ is the confidence level indicator. The participant then predicts the outcome $Z_j$ with a level of confidence $Z_j$ on a continuous scale from 0% (not at all confident) to 100% (absolutely certain). Once more a lower $BS$ implies a higher accuracy. See Juslin (1994) for a similar setup.

## Sample collection

The samples will be collected in $k=1,...,5$ rounds per each individual $i$ in $n$ participants. In each round $k$, each participant $i$ will be randomly assigned to group $T$ (tap water) or $P$ (potted water). This assignment will be carried out by a draw from the Bernoulli distribution with $p=\frac{1}{2}$. An ordered set of 100 values $0$ ($T$) and $1$ ($P$) will be generated every Monday and each participant will be allotted $0$ or $1$ on a first come first serve basis.

Each round will use a different brand of potted water - Evian, Volvic, Highland Spring, Buxton, Waitrose Essentials - and unfiltered tap water drawn from the same tap. Both will be cooled to the like temperature overnight at most. Samples will be poured from reusable vessels and tasted from disposable cups.

For each participant, we record (I) an identifier (first name), (II) the water sample tasted ($0$ or $1$), (III) the guess ($T$ or $P$), and (IV) level of confidence ([$0,100%$] from not at all confident to absolutely certain).

The participant with the most accurate score across all five rounds wins six bottles of the potted water that has done best against the tap water. In case of more than one winner, a draw from the uniform distribution decides.

The sample collection is expected to conclude by the end of November 2025. The results ought to follow by mid-December.

Caveats: Logistic difficulties preclude the application of a double blind test.

## Reference material

Agresti, A. (2019) *An Introduction to Categorical Data Analysis*, 3rd edn. Wiley.

Agresti, A. (2002) *Categorical Data Analysis*, 2nd edn. Wiley.

Brier, G.W. (1950) 'Verification of Forecasts Expressed in Terms of Probability'. *Monthly Weather Review*, Volume 78, Number 1. https://web.archive.org/web/20171023012737/https://docs.lib.noaa.gov/rescue/mwr/078/mwr-078-01-0001.pdf

Ford, C. (2023) 'Getting Started with Generalized Estimating Equations'. University of Virginia Library. https://library.virginia.edu/data/articles/getting-started-with-generalized-estimating-equations

Goldstein-Greenwood, J. (2021) 'A Brief on Brier Scores'. University of Virginia Library. https://library.virginia.edu/data/articles/a-brief-on-brier-scores

Hardin, J.W. and Hilbe, J.M. (2013) *Generalised Estimating Equations*, 2nd edn. CRC Press.

Hoessly, L. (2025) 'On misconceptions about the Brier score in binary prediction models'. arXiv. https://arxiv.org/html/2504.04906v3

Hoff, P.D. (2009) *A First Course in Bayesian Statistical Methods*. Springer.

Juslin, P. (1994) 'The Overconfidence Phenomenon as a Consequence of Informal Experimenter-Guided Selection of Almanac Items'. *Organizational Behavior and Human Decision Processes* 57, 226-246.

Marin, J.M. and Robert, C. (2014) *Bayesian Essentials with R*, 2nd edn. Springer.

Matsuura, K. (2022) *Bayesian Statistical Modeling with Stan, R, and Python*. Springer.

Robert, D. (2020) 'Five Confidence Intervals for Proportions That You Should Know About'. Towards Data Science. https://towardsdatascience.com/five-confidence-intervals-for-proportions-that-you-should-know-about-7ff5484c024f/

Stan Development Team (2024) 'Documentation'. https://mc-stan.org/docs/

StatsModels (2023) 'Generalized Estimating Equations'. https://www.statsmodels.org/stable/gee.html

## Machinery check

In [1]:
# Libraries
import math
import statistics
import numpy as np
import pandas as pd
import random
import re

from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency
from scipy.stats import norm

random.seed(42)

### Data structure and data

In [2]:
# Construct a data frame
df_dict = {
    "participant_id": []
}

for i in range(5):
    col1 = f"round{i}_sample"
    col2 = f"round{i}_prediction"
    col3 = f"round{i}_confidence"
    
    df_dict[col1] = []
    df_dict[col2] = []
    df_dict[col3] = []

df = pd.DataFrame(df_dict)

print(df.columns)

Index(['participant_id', 'round0_sample', 'round0_prediction',
       'round0_confidence', 'round1_sample', 'round1_prediction',
       'round1_confidence', 'round2_sample', 'round2_prediction',
       'round2_confidence', 'round3_sample', 'round3_prediction',
       'round3_confidence', 'round4_sample', 'round4_prediction',
       'round4_confidence'],
      dtype='object')


In [ ]:
# Generate mock data
    # Load up some names
# url = r"https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/datasets/babynamesenglandandwalestop100babynameshistoricaldata/1904to2024/historicalnames2024.xlsx"
url = r"D:\data\ons_names\historicalnames2024.xlsx"

data_g = pd.read_excel(url, sheet_name = "Table_1", skiprows = 3)
data_b = pd.read_excel(url, sheet_name = "Table_2", skiprows = 3)

data_g = data_g.drop("Rank", axis=1)
data_g = pd.melt(data_g, value_vars=data_g.columns, value_name="name")["name"]

data_b = data_b.drop("Rank", axis=1)
data_b = pd.melt(data_b, value_vars=data_b.columns, value_name="name")["name"]

names_df = pd.concat([data_g, data_b], ignore_index=True)
names_df = names_df.dropna()
names_df = names_df.drop_duplicates(ignore_index=True)

    # Construct a mock df
df_cols = df.columns
no_observations = 30

mock_df = pd.DataFrame({
    df_cols[0]: random.sample(names_df.tolist(), k=no_observations)
})

for i in range(1, len(df_cols), 3):
    col1 = [random.randint(0,1) for i in range(no_observations)]
    col2 = [random.randint(0,1) for i in range(no_observations)]
    col3 = [random.randint(0,100) / 100 for i in range(no_observations)]

    mock_df[df_cols[i]] = col1
    mock_df[df_cols[i + 1]] = col2
    mock_df[df_cols[i + 2]] = col3

mock_df.head(n=10)

,participant_id,round0_sample,round0_prediction,round0_confidence,round1_sample,round1_prediction,round1_confidence,round2_sample,round2_prediction,round2_confidence,round3_sample,round3_prediction,round3_confidence,round4_sample,round4_prediction,round4_confidence
0,Wayne,1,1,0.72,0,0,0.75,1,0,0.65,1,1,0.46,0,1,0.38
1,Hazel,0,0,0.91,0,0,0.70,0,0,0.10,0,0,0.05,1,1,0.36
2,Beatrice,1,0,0.40,0,1,0.29,0,0,0.23,1,1,0.45,0,0,0.26
3,Louis,1,0,0.27,1,1,0.75,0,1,0.08,1,0,0.26,1,1,0.55
4,Francesca,0,1,0.83,0,0,0.28,1,1,0.76,1,1,0.87,0,0,1.00
5,Jemma,0,1,0.63,1,0,0.00,1,1,0.08,0,1,0.31,0,1,0.74
6,Rachael,1,1,0.50,1,0,0.09,1,0,0.86,0,1,0.85,1,0,0.77
7,Jill,1,1,0.82,1,0,0.90,1,1,0.30,1,0,0.13,1,1,0.83
8,Jamie,1,0,0.58,1,0,0.80,1,0,0.51,0,0,0.45,1,0,0.41
9,Audrey,0,1,0.18,0,1,0.07,0,0,0.15,0,1,0.99,0,0,0.59


In [4]:
# Plug in the data
data_df = mock_df

#### Contingency tables

In [5]:
# Round dfs
df_dict = {}
df_counter = 0

for i in range(1, len(df_cols), 3):
    df_counter += 1

    samp = df_cols[i]
    pred = df_cols[i + 1]
    df_name = f"round{df_counter}"

    df_dict[df_name] = pd.DataFrame({
        "guess_t": [
            # True negatives
            sum((data_df[samp] == 0) & (data_df[pred] == 0)),
            # False negatives
            sum((data_df[samp] == 1) & (data_df[pred] == 0))
        ],
        "guess_p": [
            # False positives
            sum((data_df[samp] == 0) & (data_df[pred] == 1)),
            # True positives
            sum((data_df[samp] == 1) & (data_df[pred] == 1))
        ]
    })

    df_dict[df_name].index = ["sample_0", "sample_1"]

    print(df_dict[df_name])

          guess_t  guess_p
sample_0        6        9
sample_1        9        6
          guess_t  guess_p
sample_0        8        7
sample_1       10        5
          guess_t  guess_p
sample_0        9        8
sample_1        7        6
          guess_t  guess_p
sample_0        8        8
sample_1        8        6
          guess_t  guess_p
sample_0        6        8
sample_1        5       11


### Frequentist estimates

In [6]:
# Relative risk and odds ratio

    # Two-tailed z-score
p = 0.05
z = norm.ppf(1 - p / 2)

significance_levels = {
    "10pc": "*",
    "5pc": "**",
    "1pc": "***"
}


    # Construct a data frame
ab_results_dict = {
    "round": [],
    "relative_risk": [],
    "rr_ci_lower": [],
    "rr_ci_upper": [],
    "rr_significance": [],
    "odds_ratio": [],
    "or_ci_lower": [],
    "or_ci_upper": [],
    "or_significance": []
}

ab_results = pd.DataFrame(ab_results_dict)

    # Compute the stats
for i in range(len(df_dict)):
    df_name = list(df_dict.keys())[i]
    df_i = list(df_dict.values())[i]

    # Relative risk
    rho = (df_i.iat[0,0] / df_i.iloc[0,].sum()) / (df_i.iat[1,0] / df_i.iloc[1,].sum())
    rr = round(rho, 4)
    # RR confidence interval
    rr_se = math.sqrt((1 / df_i.iat[0,0]) - (1 / df_i.iloc[0,].sum()) + (1 / df_i.iat[1,0]) - (1 / df_i.iloc[1,].sum()))
    rr_ci = [
        round(math.exp(math.log(rho) - z * rr_se), 4),
        round(math.exp(math.log(rho) + z * rr_se), 4)
    ]
    # RR significance
    rr_significance = significance_levels[f"{int(p * 100)}pc"] if (rr_ci[0] > 1) or (rr_ci[1] < 1) else None

    # Odds ratio
    theta = (df_i.iat[0,0] / df_i.iat[0,1]) / (df_i.iat[1,0] / df_i.iat[1,1])
    or_i = round(theta, 4) 
    # OR confidence interval
    or_se = math.sqrt(sum([(1 / x) for x in df_i.to_numpy().flatten()]))
    or_ci = [
        round(math.exp(math.log(theta) - z * or_se), 4),
        round(math.exp(math.log(theta) + z * or_se), 4)
    ]
    # OR significance
    or_significance = significance_levels[f"{int(p * 100)}pc"] if (or_ci[0] > 1) or (or_ci[1] < 1) else None 
    
    # Add the row to the df
    ab_results.loc[i] = {
        "round": df_name, 
        "relative_risk": rr, 
        "rr_ci_lower": rr_ci[0], 
        "rr_ci_upper": rr_ci[1], 
        "rr_significance": rr_significance, 
        "odds_ratio": or_i, 
        "or_ci_lower": or_ci[0], 
        "or_ci_upper": or_ci[1], 
        "or_significance": or_significance
    }

ab_results.head(n=len(ab_results))

,round,relative_risk,rr_ci_lower,rr_ci_upper,rr_significance,odds_ratio,or_ci_lower,or_ci_upper,or_significance
0,round1,0.6667,0.3165,1.4042,None,0.4444,0.1031,1.9154,None
1,round2,0.8000,0.4420,1.4481,None,0.5714,0.1305,2.5026,None
2,round3,0.9832,0.5011,1.9289,None,0.9643,0.2267,4.1017,None
3,round4,0.8750,0.4488,1.7061,None,0.7500,0.1773,3.1734,None
4,round5,1.3714,0.5328,3.5304,None,1.6500,0.3696,7.3651,None


Discussion here.

### Estimating equations model

In [7]:
# Estimating equations

gee_df = data_df.loc[:, ~data_df.columns.str.contains("confidence")]
gee_df = gee_df.rename(columns=lambda c: re.sub(r"^(round\d+)_(prediction|sample)$", r"\2_\1", c)) # Flip the col name pattern for pd.wide_to_long
gee_df = gee_df.reset_index(names="row_id") # pd.wide_to_long expects a constructible unique identifier

gee_df = pd.wide_to_long(
    gee_df,
    stubnames=["prediction", "sample"],
    i=["participant_id", "row_id"],
    j="round",
    sep="_",
    suffix=r"round\d+"
).reset_index().drop(columns="row_id") 

gee_df.head()

,participant_id,round,prediction,sample
0,Wayne,round0,1,1
1,Wayne,round1,0,0
2,Wayne,round2,0,1
3,Wayne,round3,1,1
4,Wayne,round4,1,0


In [8]:
gee_df.tail()

,participant_id,round,prediction,sample
145,Elliott,round0,0,0
146,Elliott,round1,0,1
147,Elliott,round2,0,0
148,Elliott,round3,1,1
149,Elliott,round4,0,1


In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

mod = smf.gee(
    "prediction ~ sample", 
    "participant_id", 
    gee_df, 
    cov_struct=sm.cov_struct.Exchangeable(), 
    family=sm.families.Binomial()
)

res = mod.fit()

print(res.summary())

                               GEE Regression Results                              
Dep. Variable:                  prediction   No. Observations:                  150
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                   5
                      Estimating Equations   Max. cluster size:                   5
Family:                           Binomial   Mean cluster size:                 5.0
Dependence structure:         Exchangeable   Num. iterations:                     5
Date:                     Sat, 08 Nov 2025   Scale:                           1.000
Covariance type:                    robust   Time:                         09:52:44
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0909      0.219      0.415      0.678      -0.338       0.520
sample 

In [10]:
# Estimates
gee_estimates = res.params

In [11]:
# Baseline probability (s = 0)
beta_0 = gee_estimates.iloc[0]
pi_0 = 1 / (1 + math.exp(-beta_0)) # Or res.model.family.link.inverse(beta_0)

In [12]:
# Probability (s = 1)
beta_1 = gee_estimates.iloc[1]
pi_1 = 1 / (1 + math.exp(-(beta_0 + beta_1))) # Or res.model.family.link.inverse(beta_0 + beta_1)

In [13]:
# Population averaged risk ratio
rr_hat = pi_1 / pi_0

In [14]:
# RR CIs
V = res.cov_params().loc[["Intercept","sample"], ["Intercept","sample"]].to_numpy()
g = np.array([pi_0 - pi_1, 1 - pi_1])
gee_rr_var = g @ V @ g
gee_rr_ses = np.sqrt(gee_rr_var)

gee_rr_cis = (
    math.exp(math.log(rr_hat) - (z * gee_rr_ses)),
    math.exp(math.log(rr_hat) + (z * gee_rr_ses))
)

In [15]:
# Population averaged odds ratio
or_hat = math.exp(beta_1)
np.isclose(or_hat, (pi_1/(1 - pi_1)) / (pi_0/(1 - pi_0)))

np.True_

In [16]:
# OR CIs
gee_or_se = np.sqrt(res.cov_params().loc["sample", "sample"])

gee_or_cis = (
    math.exp(math.log(or_hat) - (z * gee_or_se)),
    math.exp(math.log(or_hat) + (z * gee_or_se))
)

In [17]:
# Construct a df with GEE results

gee_res_df = pd.DataFrame({
    "beta_0": [beta_0, None],
    "pi_0": [pi_0, None],
    "beta_1": [beta_1, None],
    "pi_1": [pi_1, None],
    "risk_ratio": [round(rr_hat, 3), (gee_rr_cis[0] > 1) or (gee_rr_cis[1] < 1) if "**" else "-"],
    "rr_ci_lo": [round(gee_rr_cis[0], 3), None],
    "rr_ci_hi": [round(gee_rr_cis[1], 3), None],
    "odds_ratio": [round(or_hat, 3), (gee_or_cis[0] > 1) or (gee_or_cis[1] < 1) if "**" else "-"],
    "or_ci_lo": [round(gee_or_cis[0], 3), None],
    "or_ci_hi": [round(gee_or_cis[1], 3), None]
})

gee_res_df = gee_res_df.rename(index={0: "value", 1: "significance"})
gee_res_df.head()

,beta_0,pi_0,beta_1,pi_1,risk_ratio,rr_ci_lo,rr_ci_hi,odds_ratio,or_ci_lo,or_ci_hi
value,0.0909,0.522709,-0.24177,0.462354,0.885,0.63,1.242,0.785,0.403,1.528
significance,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN


Discussion here.

### Brier score

In [18]:
# Brier score
bs_results = data_df

sample_cols = bs_results.filter(like="sample")
prediction_cols = bs_results.filter(like="prediction")
confidence_cols = bs_results.filter(like="confidence")

brier_rounds = ((sample_cols.values == prediction_cols.values).astype(int) - confidence_cols.values) ** 2
brier_score = pd.DataFrame(brier_rounds).mean(axis=1)

bs_results["brier_score"] = brier_score
bs_results = bs_results[["participant_id", "brier_score"]].sort_values(by="brier_score", ascending=True)

bs_results.head(n=10)

,participant_id,brier_score
12,Bodhi,0.04580
3,Louis,0.08238
13,Tom,0.15438
0,Wayne,0.19988
24,May,0.21278
18,Evelyn,0.21834
11,Kyle,0.21908
16,Harper,0.22650
20,Kerry,0.23138
21,Tara,0.25558


Discussion here.

### Bayesian estimates

In [19]:
import cmdstanpy



In [20]:
# Bayesian estimates
    # Libraries
import cmdstanpy
import arviz as az
import logging

    # Stan script
stan_script = """
data {
    int<lower=1> K; // {treatment, control}
    array[K] int<lower=0> n; // Trials per group
    array[K] int<lower=0> y; // Successes per group
}
parameters {
    vector<lower=0, upper=1>[K] p;
}
model {
    // Jeffreys prior
    p ~ beta(0.5, 0.5);

    // Binomial likelihood
    y ~ binomial(n, p);
}
generated quantities {
    real risk_ratio = p[1] / p[2];
    real odds_ratio = (p[1] / (1 - p[1])) / (p[2] / (1 - p[2]));
}
"""

    # Write the Stan model into a file
with open('beta_binomial_model.stan', 'w') as f:
    f.write(stan_script)
    # Compile the model
mod = cmdstanpy.CmdStanModel(stan_file='beta_binomial_model.stan')

    # df
bayes_df = pd.DataFrame({
    "round": [],
    "p_1": [],
    "p_2":[],
    "risk_ratio": [],
    "rr_lower": [],
    "rr_upper": [],
    "rr_significance": [],
    "odds_ratio": [],
    "or_lower": [],
    "or_upper": [],
    "or_significance": []
})

    # Suppress the cmdstanpy MCMC readout
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)

for name_i, contingency_table_i in df_dict.items():
    idx = int(re.search("\\d{1}", name_i).group())
    ct = contingency_table_i.to_numpy()

    stan_data = {
        "K": 2, # {treatment, control}
        "n": [sum(ct[0]), sum(ct[1])], # Trials per group
        "y": [ct[0,0], ct[1,0]] # Successes per group
    }

    fit = mod.sample(
        data = stan_data,
        chains = 6,
        parallel_chains = 6,
        iter_warmup = int(4e2), # 4e3
        iter_sampling = int(16e2), # 16e3
        seed = 42,
        refresh = None, # Suppresses progress updates
        show_progress = False # Suppresses Stan messages
    )

    inference_data = az.from_cmdstanpy(
        posterior = fit,
        posterior_predictive = None,
        coords={'group': ['treatment', 'control']},
        dims={'p': ['group']}
    )
    
    print(f"\nSummary statistics and diagnostics for {name_i}:\n")
    print(az.summary(inference_data))
    # print(fit.summary()[['Mean', 'StdDev', 'R_hat']])
    
    # Extract posterior samples
    posterior_draws_df = fit.draws_pd()
    # posterior_draws = fit.draws() # numpy array

    # Posterior summaries
    draws_df = posterior_draws_df[["p[1]", "p[2]", "risk_ratio", "odds_ratio"]]
    rr_q = draws_df["risk_ratio"].quantile([0.025, 0.5, 0.975]).values
    or_q = draws_df["odds_ratio"].quantile([0.025, 0.5, 0.975]).values

    # Add the row to the df
    bayes_df.loc[idx] = {
        "round": name_i,
        "p_1": round(draws_df["p[1]"].median(), 3),
        "p_2":round(draws_df["p[2]"].median(), 3),
        "risk_ratio": round(draws_df["risk_ratio"].median(), 3),
        "rr_lower": round(rr_q[0], 3),
        "rr_upper": round(rr_q[2], 3),
        "rr_significance": "**" if (rr_q[0] > 1) or (rr_q[2] < 1) else None,
        "odds_ratio": round(draws_df["odds_ratio"].median(), 3),
        "or_lower": round(or_q[0], 3),
        "or_upper": round(or_q[2], 3),
        "or_significance": "**" if (or_q[0] > 1) or (or_q[2] < 1) else None
    }
    
bayes_df.head()

09:52:44 - cmdstanpy - INFO - compiling stan file C:\Users\rosec\OneDrive\ACADEM~1\coding\Jupyter\beta_binomial_model.stan to exe file C:\Users\rosec\OneDrive\Academic folder\coding\Jupyter\beta_binomial_model.exe
09:52:53 - cmdstanpy - INFO - compiled model executable: C:\Users\rosec\OneDrive\Academic folder\coding\Jupyter\beta_binomial_model.exe



Summary statistics and diagnostics for round1:

               mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
p[treatment]  0.408  0.119   0.192    0.631      0.001    0.001    9223.0   
p[control]    0.592  0.118   0.377    0.813      0.001    0.001    9000.0   
risk_ratio    0.722  0.276   0.258    1.225      0.003    0.004    9046.0   
odds_ratio    0.592  0.487   0.034    1.410      0.006    0.011    9159.0   

              ess_tail  r_hat  
p[treatment]    5950.0    1.0  
p[control]      6513.0    1.0  
risk_ratio      6288.0    1.0  
odds_ratio      6163.0    1.0  

Summary statistics and diagnostics for round2:

               mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
p[treatment]  0.530  0.120   0.311    0.755      0.001    0.001    8665.0   
p[control]    0.657  0.114   0.457    0.877      0.001    0.001    8867.0   
risk_ratio    0.836  0.258   0.392    1.320      0.003    0.004    8193.0   
odds_ratio    0.748  0.619   0.073    1.821     

,round,p_1,p_2,risk_ratio,rr_lower,rr_upper,rr_significance,odds_ratio,or_lower,or_upper,or_significance
1,round1,0.405,0.597,0.684,0.304,1.376,None,0.456,0.104,1.904,None
2,round2,0.532,0.663,0.805,0.422,1.423,None,0.575,0.128,2.395,None
3,round3,0.527,0.540,0.978,0.497,1.989,None,0.953,0.225,3.895,None
4,round4,0.501,0.571,0.882,0.445,1.764,None,0.754,0.175,3.127,None
5,round5,0.430,0.316,1.351,0.535,3.649,None,1.632,0.374,7.276,None


Discussion here.